**Constantes**

In [24]:
import math

# Diametro Externo
diameter = 0.40 # metros

#Espessura do aço
steel_thickness = 0.02

# Número pi
pi_number = math.pi

# Comprimento da tubulação (em metros)
length = 40

# Contribuição da rigidez à flexão do concreto e revestimento (expressa como fração de EI_steel)
csf = 0  # Neste caso, não há contribuição adicional de concreto ou revestimento

# Rigidez à flexão do aço (EI_steel), dada em N.m²
ei = 8.946 * (10 ** 7 )

# Constante de Temperatura
temperature_coefficient = 0.00017  # Coeficiente de expansão térmica

# Modulo de Elasticidade
young_modulus = 207 * (10**9)  # Módulo de elasticidade

# Coeficiente de Poisson
poisson_coefficient_steel = 0.3
poisson_coefficient_soil = 0.45

# Massa Especifica
specific_mass_steel = 7850 # Kg/m³
specific_mass_water = 1027 # Kg/m³

# Constante Gravitacional
gravity = 10 # m/s²

me = 316.484 # Kg/m

**Força Axial Efetiva**

In [26]:
import math

# Tensão axial aplicada durante a instalação da tubulação (N)
effective_lay_tension = 90000

# Diferença de pressão interna (assumida como zero neste caso)
internal_pressure_diff = 0

# Cálculo da área da seção transversal da tubulação (m²)
cross_sectional_area = pi_number * (diameter ** 2) / 4

# Diferença de temperatura entre a instalação e a operação (assumida como zero neste caso)
temperature_diff = 0

# **Cálculo da força axial efetiva**
effective_axial_force = (effective_lay_tension -  # Tensão axial de instalação
                        internal_pressure_diff * cross_sectional_area *  # Termo da pressão interna
                        (1 - 2 * poisson_coefficient_steel) -  # Ajuste devido ao efeito de Poisson
                        cross_sectional_area * young_modulus * temperature_diff)  # Termo da variação térmica

print('Força Axial Efetiva: ' + str(effective_axial_force))



Força Axial Efetiva: 90000.0


**Rigidez do Solo**

In [19]:

# Effetive Lenght
area_pipe = pi_number * (((diameter/2)**2) - (((diameter/2)- steel_thickness)**2))
densidade_steel = 7850 #kg/m³
ms =  densidade_steel * area_pipe * 40 # mass of the pipe
densidade_water = 1027 #kg/m³
area_water = pi_number * ((diameter/2) ** 2)
m = densidade_water * area_water * 40  #  mass of the displaced water

ds_per_d = ms/m

cv = 600000 # boundary condition coefficient (vertical dynamic stiffness)
cl = 500000 # boundary condition coefficient (lateral dynamic stiffness)

## Definição dos parâmetros de rigidez do solo para argila muito mole (Clay Very Soft)
kvs = 75000 # N/m/m - Rigidez estática vertical do solo por unidade de comprimento
kv = (cv/(1-poisson_coefficient_soil))*((2/3)*(ds_per_d) + 1/3)*math.sqrt(diameter) # N/m/m - Rigidez dinâmica vertical do solo por unidade de comprimento
kl = (cl*(1+poisson_coefficient_soil))*((2/3)*(ds_per_d) + 1/3)*math.sqrt(diameter) # N/m/m - Rigidez dinâmica lateral do solo por unidade de comprimento

print('Rigidez dinâmica vertical do solo: ' + str(kv))
print('Rigidez dinâmica lateral do solo: ' + str(kl))

Rigidez dinâmica vertical do solo: 897989.4426679793
Rigidez dinâmica lateral do solo: 596788.8171064279


**Carga Crítica de Flambagem**




In [6]:
def effetive_length(k: int, length: int, csf: int, stiffness: float):

  if k == kv:
    print('Rigidez dinâmica vertical do solo')

  elif k == kl:
    print('Rigidez dinâmica lateral do solo')

  else:
    print('Rigidez estática vertical do solo')

  # Cálculo do logaritmo do valor adimensional que influencia o comprimento efetivo
  value_log = k * (length ** 4) / ((1 + csf) * stiffness)
  print('value_log: ' + str (value_log))

  beta = math.log10(value_log)  # Cálculo do logaritmo de base 10 do valor obtido
  print('beta: ' + str(beta))  # Exibe o valor de beta

  # Cálculo da razão entre o comprimento efetivo e o comprimento total da tubulação
  ratio_length_eff = 4.73 / ((-0.066 * (beta ** 2)) + (1.02 * beta) + 0.63)

  # Cálculo do comprimento efetivo da tubulação
  length_eff = ratio_length_eff * length

  print('Comprimento Efetivo: ' + str(length_eff))  # Exibe o comprimento efetivo da tubulação

  # Coeficiente de condição de contorno
  c2 = 4

  # Cálculo da carga crítica de flambagem (Pcr)
  pcr = (1 + csf) * c2 * (pi_number ** 2) * stiffness / (length_eff ** 2)

  print('Carga Crítica de Flambagem: ' + str(pcr))  # Exibe o valor da carga crítica de flambagem

  # Razão entre uma carga específica (90000 N) e a carga crítica de flambagem
  print('Razão entre Força Axial Efetiva e Carga Crítica de Flambagem: ' + str(effective_axial_force / pcr))  # Exibe a relação entre a carga axial efetiva e a carga crítica de flambagem
  print('')

  return length_eff, pcr

length_eff_vertical_dynamic, pcr_vertical_dynamic = effetive_length(k=kv, length=length, csf=csf, stiffness=ei)
length_eff_lateral_dynamic, pcr_lateral_dynamic = effetive_length(k=kl, length=length, csf=csf, stiffness=ei)
length_eff_vertical_static, pcr_vertical_static = effetive_length(k=kvs, length=length, csf=csf, stiffness=ei)


Rigidez dinâmica vertical do solo
value_log: 25696.992770288703
beta: 4.409882302331834
Comprimento Efetivo: 49.21221618037678
Carga Crítica de Flambagem: 1458286.2614672505
Razão entre Força Axial Efetiva e Carga Crítica de Flambagem: 0.06171627778310601

Rigidez dinâmica lateral do solo
value_log: 17077.793111921033
beta: 4.232431748013428
Comprimento Efetivo: 50.25511640808623
Carga Crítica de Flambagem: 1398389.2089517515
Razão entre Força Axial Efetiva e Carga Crítica de Flambagem: 0.06435976438023648

Rigidez estática vertical do solo
value_log: 2146.2105969148224
beta: 3.3316723348669113
Comprimento Efetivo: 57.408083782216394
Carga Crítica de Flambagem: 1071624.2403273375
Razão entre Força Axial Efetiva e Carga Crítica de Flambagem: 0.08398466235936271



**Deflexão Estática**

In [27]:
# Coeficiente c2 de condição de contorno
c6 = 1/384

# Peso do tubo por unidade de comprimento
pipe_weight =  specific_mass_steel * gravity * area_pipe

# Cálculo do Empuxo
empuxo = specific_mass_water * gravity * area_water

# Solicitacao Axial Efetiva
seff = effective_axial_force

# Peso Submerso por Unidade de Comprimento
q = pipe_weight - empuxo

# Cálculo da Deflexão Estática
deflection = c6 * ((q * (length_eff_vertical_static**4))/(ei * (1 + csf))) * (1 / (1 + seff/pcr_vertical_static))
deflection_vertical_dynamic = c6 * ((q * (length_eff_vertical_dynamic**4))/(ei * (1 + csf))) * (1 / (1 + seff/pcr_vertical_dynamic))

print('Deflexão: ' + str(deflection))
print('Razão entre deflexão e diâmetro: '+ str(deflection/diameter))

Deflexão: 0.1701705120336292
Razão entre deflexão e diâmetro: 0.425426280084073


**Amplitude máxima de tensão**

In [8]:
# Coeficiente C4 de condição de contorno
length_eff_list = [length_eff_vertical_dynamic, length_eff_lateral_dynamic, length_eff_vertical_static]

for length_eff in length_eff_list:

  c4_midspan = 8.6
  c4_shouder = 14.1 * ((length/length_eff)**2)
  c4 = [c4_midspan, c4_shouder]

  if length_eff == length_eff_vertical_dynamic:
    name_length_eff = 'Rigidez dinâmica vertical do solo'

  elif length_eff == length_eff_lateral_dynamic:
    name_length_eff = 'Rigidez dinâmica lateral do solo'

  else:
     name_length_eff ='Rigidez estática vertical do solo'

  for c in c4:

    if c == c4_midspan:
      name_c = 'midspan'

    else:
      name_c = 'shouder'

    radius_extern = diameter / 2
    radius_intern = (diameter - 2*steel_thickness)/2
    radius_list = [radius_extern, radius_intern]

    for radius in radius_list:

      if radius == radius_extern:
        print(name_length_eff + ', ' + name_c + ', ' +' raio externo')

      else:
        print(name_length_eff + ', ' + name_c +', ' + 'raio interno')

      amplitude_max_inline = 2 * c * (1 + csf) * diameter * young_modulus * radius / (length_eff ** 2)
      #amplitude_max_crossflow = 2 * c * (1 + csf) * diameter * young_modulus * radius / (length_eff ** 2)

      print('Amplitude (MPa): ' + str(amplitude_max_inline/1000000))
      #print('Amplitude cross-flow (MPa): ' + str(amplitude_max_crossflow/1000000))
      print('')

Rigidez dinâmica vertical do solo, midspan,  raio externo
Amplitude (MPa): 117.60964338841694

Rigidez dinâmica vertical do solo, midspan, raio interno
Amplitude (MPa): 105.84867904957525

Rigidez dinâmica vertical do solo, shouder,  raio externo
Amplitude (MPa): 127.39070203315512

Rigidez dinâmica vertical do solo, shouder, raio interno
Amplitude (MPa): 114.65163182983962

Rigidez dinâmica lateral do solo, midspan,  raio externo
Amplitude (MPa): 112.77899307475539

Rigidez dinâmica lateral do solo, midspan, raio interno
Amplitude (MPa): 101.50109376727987

Rigidez dinâmica lateral do solo, shouder,  raio externo
Amplitude (MPa): 117.14082820147966

Rigidez dinâmica lateral do solo, shouder, raio interno
Amplitude (MPa): 105.42674538133171

Rigidez estática vertical do solo, midspan,  raio externo
Amplitude (MPa): 86.42565460671165

Rigidez estática vertical do solo, midspan, raio interno
Amplitude (MPa): 77.78308914604048

Rigidez estática vertical do solo, shouder,  raio externo
Amp

In [9]:
amplitude_tensao_crossflow = (127.391 + 114.652)/2
print('amplitude_tensao_crossflow: '+ str(amplitude_tensao_crossflow))

amplitude_tensao_inline = (117.141 + 105.427)/2
print('amplitude_tensao_inline: '+ str(amplitude_tensao_inline))

amplitude_tensao_crossflow: 121.0215
amplitude_tensao_inline: 111.284


**Frequência Natural Fundamental**

In [10]:
# Boundary condition coefficients
c1_inline, c1_crossflow = 3.56, 3.56
c3_inline, c3_crossflow = 0, 0.4

print ()
f_inline = c1_inline * math.sqrt(1 + csf) * math.sqrt(ei/(me * (length_eff_lateral_dynamic ** 4)) * (1 + (seff/pcr_lateral_dynamic) + c3_inline * (deflection/diameter) ** 2))
f_crossflow = c1_crossflow * math.sqrt(1 + csf) * math.sqrt(ei/(me * (length_eff_vertical_dynamic ** 4)) * (1 + (seff/pcr_vertical_dynamic) + c3_crossflow * (deflection/diameter) ** 2))

print('Frequência na direção in-line (Hz) :'+ str(f_inline))
print('Frequência na direção cross-flow (Hz) :'+ str(f_crossflow))


Frequência na direção in-line (Hz) :0.7731650662621364
Frequência na direção cross-flow (Hz) :0.8323090555616094
